In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
  
df_articles = pd.read_csv('shared_articles.csv')
df_articles.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [12]:
print(df_articles.shape)
df_articles.isna().sum()

(3122, 13)


timestamp             0
eventType             0
contentId             0
authorPersonId        0
authorSessionId       0
authorUserAgent    2442
authorRegion       2442
authorCountry      2442
contentType           0
url                   0
title                 0
text                  0
lang                  0
dtype: int64

In [13]:
df_articles.authorUserAgent.fillna('unknown', inplace=True)
df_articles.authorRegion.fillna('unknown', inplace=True)
df_articles.authorCountry.fillna('unknown', inplace=True)

      
# Very important step
df_articles.reset_index(inplace=True)
      
print(df_articles.isna().sum(), '\n')
df_articles.shape

index              0
timestamp          0
eventType          0
contentId          0
authorPersonId     0
authorSessionId    0
authorUserAgent    0
authorRegion       0
authorCountry      0
contentType        0
url                0
title              0
text               0
lang               0
dtype: int64 



/var/folders/89/ddk09q_13q5dxg5njh_qm7p40000gn/T/ipykernel_31958/2404978803.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_articles.authorUserAgent.fillna('unknown', inplace=True)
/var/folders/89/ddk09q_13q5dxg5njh_qm7p40000gn/T/ipykernel_31958/2404978803.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

(3122, 14)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
          
# Create a TfidfVectorizer and Remove stopwords
tfidf = TfidfVectorizer(stop_words='english')
        
# Fit and transform the data to a tfidf matrix
tfidf_matrix = tfidf.fit_transform(df_articles['text'])
        
# Print the shape of the tfidf_matrix
print(tfidf_matrix.shape)
        
# Preview the matrix by placing it into a DataFrame (which we won't need later)
df_tfidf = pd.DataFrame(tfidf_matrix.T.todense(), index=tfidf.get_feature_names_out(), columns=df_articles['title'])
df_tfidf.iloc[2221:2226]
      
# Output
# (8803, 18891)

(3122, 72353)


title,"Ethereum, a Virtual Currency, Enables Transactions That Rival Bitcoin's","Ethereum, a Virtual Currency, Enables Transactions That Rival Bitcoin's",Bitcoin Future: When GBPcoin of Branson Wins Over USDcoin of Trump,Google Data Center 360° Tour,"IBM Wants to ""Evolve the Internet"" With Blockchain Technology",IEEE to Talk Blockchain at Cloud Computing Oxford-Con - CoinDesk,Banks Need To Collaborate With Bitcoin and Fintech Developers,Blockchain Technology Could Put Bank Auditors Out of Work,Why Decentralized Conglomerates Will Scale Better than Bitcoin - Interview with OpenLedger CEO - Bitcoin News,The Rise And Growth of Ethereum Gets Mainstream Coverage,...,Life Beyond Email: Chatbot Marketing,Another option for file sharing,Gartner Reprint,Command-line utilities,Swift Top 10 Articles For The Past Year (v.2017),"Conheça a Liga IoT, plataforma de inovação aberta que irá acelerar projetos ligados a Internet das Coisas",Amazon takes on Skype and GoToMeeting with its Chime video conferencing app,Code.org 2016 Annual Report,"JPMorgan Software Does in Seconds What Took Lawyers 360,000 Hours",The 2017 Acquia Partners of the Year
980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98347107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98360656,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity between each movie description
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
      
# For easier viewing, put it in a dataframe
pd.DataFrame(cosine_sim)

df_results = pd.DataFrame(cosine_sim, columns=df_articles['contentId'], index=df_articles['contentId']) #.sort_values(by=[0], ascending=False)

df_results.to_csv('news_content_filtering_results.csv')

In [10]:
df_sorted = pd.DataFrame(cosine_sim).sort_values(by=[0], ascending=False)

for id in df_sorted.index[1:6]:
    print(id, '\t', df_articles.loc[id, 'title'])
        
display(df_sorted)

1 	 Ethereum, a Virtual Currency, Enables Transactions That Rival Bitcoin's
9 	 The Rise And Growth of Ethereum Gets Mainstream Coverage
525 	 For Blockchain VCs, the Time for Ethereum Investments Has Come - CoinDesk
184 	 Ethereum and Bitcoin Are Market Leaders But Not Competitors
83 	 Microsoft Adds Ethereum to Windows Platform For Over 3 Million Developers - CoinDesk


,0,1,2,3,4,5,6,7,8,9,...,3112,3113,3114,3115,3116,3117,3118,3119,3120,3121
0,1.000000,1.000000,0.029999,0.016192,0.234378,0.102207,0.186021,0.150986,0.182221,0.865994,...,0.043244,0.024061,0.007743,0.023067,0.011399,0.003296,0.021317,0.051052,0.090121,0.020183
1,1.000000,1.000000,0.029999,0.016192,0.234378,0.102207,0.186021,0.150986,0.182221,0.865994,...,0.043244,0.024061,0.007743,0.023067,0.011399,0.003296,0.021317,0.051052,0.090121,0.020183
9,0.865994,0.865994,0.031538,0.015807,0.186033,0.066957,0.145716,0.119134,0.149767,1.000000,...,0.035278,0.017967,0.007885,0.015118,0.013960,0.002543,0.015953,0.042678,0.077155,0.014824
525,0.573491,0.573491,0.025405,0.012065,0.246473,0.124257,0.176664,0.145108,0.176852,0.575536,...,0.048840,0.018346,0.015196,0.019146,0.011161,0.039552,0.014295,0.021376,0.115521,0.018514
184,0.561477,0.561477,0.026309,0.014411,0.260029,0.107196,0.254204,0.177274,0.241549,0.542963,...,0.031366,0.019723,0.003328,0.015546,0.012231,0.000914,0.018423,0.014359,0.039569,0.003485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368,0.000000,0.000000,0.003515,0.000000,0.001046,0.000000,0.002118,0.000000,0.002559,0.000000,...,0.000000,0.006711,0.000000,0.000861,0.000000,0.138418,0.000000,0.003298,0.000909,0.004019
1814,0.000000,0.000000,0.000000,0.000000,0.000281,0.000000,0.001992,0.000000,0.000445,0.000000,...,0.001519,0.000584,0.000000,0.000762,0.001337,0.079232,0.000913,0.000143,0.000427,0.000000
1815,0.000000,0.000000,0.001065,0.034515,0.001317,0.000000,0.001166,0.000000,0.005037,0.000000,...,0.004029,0.008507,0.074037,0.000000,0.000000,0.121568,0.000000,0.003384,0.001404,0.003532
449,0.000000,0.000000,0.013042,0.006596,0.001987,0.001982,0.003672,0.000000,0.003802,0.000000,...,0.091421,0.008303,0.000000,0.000726,0.001343,0.190069,0.010380,0.001703,0.018219,0.000000


In [8]:
def get_recommendations(contentId, sim_matrix, n=10, messages=True):
        
    # Get the pairwise similarity scores of all articles with that article
    sim_scores = list(enumerate(sim_matrix[contentId]))
        
    # Sort the items based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
    # Get the scores of the n most similar items; start at 1 so that it skips itself
    top_similar = sim_scores[1:n+1]
        
    # Put the recommended item indices and similarity scores together in a dictionary using comprehension
    rec_dict = {i[0]:i[1] for i in top_similar}
        
    if messages:
        print(f"The top recommended item IDs are: {list(rec_dict.keys())}")
        print(f"Their similarity scores are:\t  {list(rec_dict.values())}")
        
    # Return the top n most similar items
    return rec_dict

In [9]:
# Change this value to any title you'd like to get recommendations
title = "Google Data Center 360° Tour"
        
# Check if the title is valid; if not, suggest alternatives and use the last one for recommendations
if title in df_articles['title'].to_list():
  id = df_articles.index[df_articles['title']==title][0] # Convert the title to an index (i.e. item ID)
else:
  print(f"\"{title}\" is not in the data set. Try one of these:\n")
  for row in df_articles.sample(n=10).itertuples():  # Get a random 10 titles
    id = row[0]
    title = row.title
    print(f'\t{title}')
        
print(f"\nIf you like \"{title},\" then you may also like:\n")
        
# Call the function and return the dictionary; print out the dictionary if you want to see what it is
recommend_dict = get_recommendations(id, cosine_sim, n=10, messages=False)
        
# Add the dictionary to a new DataFrame; this isn't necessary, but it helps to see what articles are recommended
df_similarity = pd.DataFrame(data=recommend_dict.values(), columns=['similarity'], index=recommend_dict.keys())
        
# Create a subset of the original df DataFrame with only the recommended articles
df_recommendations = df_articles.loc[df_articles.index.isin(recommend_dict.keys()), ['title',	'text']]
        
# Join the original df results with the recommended article similarity scores so that we can sort the list and view it
df_recommendations.join(df_similarity).sort_values(by=['similarity'], ascending=False)


If you like "Google Data Center 360° Tour," then you may also like:



,title,text,similarity
142,Google shares data center security and design ...,GCP NEXT 2016 - San Francisco - Attendees at G...,0.305122
676,This year's Founders' Letter,"Every year, Larry and Sergey write a Founders'...",0.278009
248,Google Cloud Platform: The smart person's guid...,From its humble beginning with Google App Engi...,0.274672
906,YouTube's New Messenger Means You'll Never Hav...,We've all had it happen: You're talking with a...,0.272787
547,Top 5 GCP NEXT breakout sessions on YouTube (s...,Couldn't make it to GCP NEXT last month? Lucky...,0.260838
884,Google I/O 2016 Preview: A Chrome/Android merg...,Google I/O is Google's annual launch party whe...,0.251773
958,"Google I/O 2016 preview: Android N, Android VR...",What to expect from the biggest Google confere...,0.219327
2885,[Tools] How to Record your Desktop Screen with...,This step-by-step guide explains how you can r...,0.213211
962,"Google I/O 2016 Preview: Machine Learning, Vir...",The democratization of machine learning contin...,0.212583
79,Google admits original enterprise cloud strate...,"Image: Andy Wolber/TechRepublic Meet Google, a...",0.211202


In [ ]:
import joblib

joblib.dump(cosine_sim, 'cosine_similarity_matrix.sav')

joblib.dump(tfidf, 'tfidf_vectorizer.sav')
